# MMD 2024, Collaborative Filtering on Google Colab
This notebook sets up the enviroment and runs CF experiments on Google Colab.





In [1]:
# Clone the repository to local runtime

private = False
if private:
    # Private repository, requires authentication
    from google.colab import userdata
    pat = userdata.get('github_pat')
    project = '24WS-mmd-code-priv'
else:
    pat = ''
    project = '24WS-mmd-code-public'

In [2]:
!git clone https://{pat}@github.com/aip-hd-tea/{project}.git

Cloning into '24WS-mmd-code-public'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 30 (delta 8), reused 27 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 8.12 KiB | 2.03 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [128]:
# Import the repository code
import sys
sys.path.insert(0,f"/content/{project}")

import rec_sys.data_util as cfd

# After edits of cf_algorithms_to_complete.py:
# 1. Rename the file rec_sys.cf_algorithms_to_complete.py to rec_sys.cf_algorithms.py
# 2. Restart the runtime (Runtime -> Restart the session); possibly not needed
# 3. Swap the comments in the next two lines, so that cf_algorithms is imported as cfa
#import rec_sys.cf_algorithms_to_complete as cfa
import rec_sys.cf_algorithms_new as cfa
# 4. Re-run all cells
# 5. If your changes are correct, you will see a long
#    printout of recommendations for MovieLens dataset (last cell)

In [129]:
# Load or set the configuration
#from rec_sys.cf_config import config

import dataclasses
@dataclasses.dataclass
class config:
    max_rows: int = int(1e5)
    dowload_url: str = "https://files.grouplens.org/datasets/movielens/ml-25m.zip"
    download_dir: str = "/content/"
    unzipped_dir: str = download_dir + "ml-25m/"
    file_path: str = download_dir + "ml-25m/ratings.csv"


In [101]:
pip install pympler

cf_algorithms.py

In [ ]:
# Artur Andrzejak, October 2024
# Algorithms for collaborative filtering

from scipy.sparse import csr_matrix, issparse
import numpy as np
from pympler import asizeof
import tensorflow_datasets as tfds

def complete_code(message):
    raise Exception(f"Please complete the code: {message}")
    return None


def center_and_nan_to_zero(matrix, axis=0):
    """ Center the matrix and replace nan values with zeros"""
    # Compute along axis 'axis' the mean of non-nan values
    # E.g. axis=0: mean of each column, since op is along rows (axis=0)
    means = np.nanmean(matrix, axis=axis)
    # Subtract the mean from each axis
    matrix_centered = matrix - means
    return np.nan_to_num(matrix_centered)


def cosine_sim(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))


def fast_cosine_sim(utility_matrix, vector, axis=0):
    """ Compute the cosine similarity between the matrix and the vector"""
    # Compute the norms of each column
    norms = np.linalg.norm(utility_matrix, axis=axis)
    um_normalized = utility_matrix / norms
    # Compute the dot product of transposed normalized matrix and the vector
    dot = np.dot(um_normalized.T,vector)
    # Scale by the vector norm
    scaled = dot / np.linalg.norm(vector)
    return scaled

#excercise 2
def centered_cosine_sim(vec1, vec2):
    """Compute centered cosine similarity between two sparse vectors, handling NaNs as zeroes."""
    # Ensure data is in float format to avoid type issues
    vec1 = csr_matrix(vec1).astype(np.float64)
    vec2 = csr_matrix(vec2).astype(np.float64)

    # Center each vector by subtracting the mean of non-NaN values
    mean_vec1 = np.nanmean(vec1.data)
    mean_vec2 = np.nanmean(vec2.data)

    vec1_centered = vec1.copy()
    vec2_centered = vec2.copy()
    vec1_centered.data -= mean_vec1
    vec2_centered.data -= mean_vec2

    # Replace any NaN entries with zero in the centered vectors
    vec1_centered.data = np.nan_to_num(vec1_centered.data)
    vec2_centered.data = np.nan_to_num(vec2_centered.data)

    # Compute dot product and norms for similarity
    dot_product = vec1_centered.dot(vec2_centered.T).toarray()[0][0]
    norm1 = np.sqrt(vec1_centered.multiply(vec1_centered).sum())
    norm2 = np.sqrt(vec2_centered.multiply(vec2_centered).sum())

    # Calculate cosine similarity, handling division by zero
    if norm1 != 0 and norm2 != 0:
        return dot_product / (norm1 * norm2)
    else:
        return 0.0


def fast_centered_cosine_sim(utility_matrix, vector, axis=0):
    """
    Compute centered cosine similarity between each row or column of a sparse matrix and a sparse vector.
    If axis=0, computes row-wise similarity. If axis=1, computes column-wise similarity.
    """
    if not issparse(utility_matrix):
        utility_matrix = csr_matrix(utility_matrix)
    if not issparse(vector):
        vector = csr_matrix(vector)

    # Convert utility matrix to dense format for mean subtraction
    utility_dense = utility_matrix.toarray()
    if axis == 0:
        row_means = np.nanmean(utility_dense, axis=1).reshape(-1, 1)
        matrix_centered = utility_dense - row_means  # Centered utility matrix
    else:
        col_means = np.nanmean(utility_dense, axis=0)
        matrix_centered = utility_dense - col_means  # Centered utility matrix

    # Convert centered matrix back to sparse format
    matrix_centered = csr_matrix(np.nan_to_num(matrix_centered))

    # Center the vector by converting to dense for mean subtraction, then back to sparse
    vector_dense = vector.toarray()
    vector_mean = np.nanmean(vector_dense)
    vector_centered = csr_matrix(vector_dense - vector_mean)  # Centered vector in sparse format

    # Ensure vector shape matches matrix dimension
    if axis == 0:
        vector_centered = vector_centered.toarray().flatten()[:matrix_centered.shape[1]]  # Match column count
    else:
        vector_centered = vector_centered.toarray().reshape(-1, 1)[:matrix_centered.shape[0]]  # Match row count

    # Debug shapes before dot product
    print("matrix_centered shape:", matrix_centered.shape)
    print("vector_centered shape after adjustment:", vector_centered.shape)

    # Check compatibility for dot product
    if axis == 0 and matrix_centered.shape[1] != vector_centered.shape[0]:
        raise ValueError(f"Dimension mismatch: matrix columns {matrix_centered.shape[1]} vs. vector {vector_centered.shape[0]}")
    elif axis == 1 and matrix_centered.shape[0] != vector_centered.shape[0]:
        raise ValueError(f"Dimension mismatch: matrix rows {matrix_centered.shape[0]} vs. vector {vector_centered.shape[0]}")

    # Compute the dot product and norms
    dot_product = matrix_centered.dot(vector_centered)
    matrix_norms = np.sqrt(matrix_centered.multiply(matrix_centered).sum(axis=1)).A1  # Row norms
    vector_norm = np.linalg.norm(vector_centered)  # Vector's norm

    # Calculate cosine similarity with zero-safe division
    similarity = np.divide(dot_product, matrix_norms * vector_norm, out=np.zeros_like(dot_product), where=(matrix_norms != 0) & (vector_norm != 0))
    return similarity


#exercise 3

def rate_all_items(orig_utility_matrix, user_index, neighborhood_size):
    print(f"\n>>> CF computation for UM w/ shape: {orig_utility_matrix.shape}, user_index: {user_index}, neighborhood_size: {neighborhood_size}\n")

    clean_utility_matrix = center_and_nan_to_zero(orig_utility_matrix)
    user_col = clean_utility_matrix[:, user_index]
    similarities = fast_centered_cosine_sim(clean_utility_matrix, user_col, axis=0)

    def rate_one_item(item_index):
        # If the user has already rated the item, return the rating
        if not np.isnan(orig_utility_matrix[item_index, user_index]):
            return orig_utility_matrix[item_index, user_index]

        # Find the indices of users who rated the item
        users_who_rated = np.where(~np.isnan(orig_utility_matrix[item_index, :]))[0]

        # If no users have rated this item, return NaN or a default rating
        if users_who_rated.size == 0:
            return np.nan  # or a default rating, e.g., `return 0.0`

        # Select top neighbors based on similarities
        best_among_who_rated = np.argsort(similarities[users_who_rated])[-neighborhood_size:]
        best_among_who_rated = users_who_rated[best_among_who_rated]
        best_among_who_rated = best_among_who_rated[~np.isnan(similarities[best_among_who_rated])]

        if best_among_who_rated.size > 0:
            rating_of_item = np.dot(similarities[best_among_who_rated], orig_utility_matrix[item_index, best_among_who_rated]) / np.sum(np.abs(similarities[best_among_who_rated]))
        else:
            rating_of_item = np.nan  # if no neighbors are valid, return NaN

        print(f"item_idx: {item_index}, neighbors: {best_among_who_rated}, rating: {rating_of_item}")
        return rating_of_item

    ratings = list(map(rate_one_item, range(orig_utility_matrix.shape[0])))
    return ratings


#exercise 4

# Set up the configuration for the dataset load
#config = ConfigLf()

# Define the function to create sparse data structures based on TF data
def create_sparse_structures_with_tf():
    # Initialize dictionaries to accumulate data
    rated_by = {}
    user_data = {}

    # Load the dataset using TensorFlow Datasets
    ratings, user_ids_voc, movie_ids_voc = load_movielens_tf(config)

    # Process each entry in the dataset to populate rated_by and user_data
    for record in ratings:
        user_id = record['user_id'].numpy()
        movie_id = record['movie_id'].numpy()
        rating = record['user_rating'].numpy()

        # Populate rated_by dictionary
        if movie_id not in rated_by:
            rated_by[movie_id] = []
        rated_by[movie_id].append(user_id)

        # Populate user_data for user_col later
        if user_id not in user_data:
            user_data[user_id] = {}
        user_data[user_id][movie_id] = rating

    # Convert rated_by to list format
    num_items = max(rated_by.keys()) + 1
    rated_by_list = [rated_by.get(item, []) for item in range(num_items)]

    # Create user_col as sparse vectors
    num_users = max(user_data.keys()) + 1
    user_col = [
        csr_matrix((list(user_data[u].values()), ([0] * len(user_data[u]), list(user_data[u].keys()))), shape=(1, num_items))
        if u in user_data else csr_matrix((1, num_items)) for u in range(num_users)
    ]

    return rated_by_list, user_col

    #exercise 5

import numpy as np
import psutil  # For memory usage monitoring
from scipy.sparse import csr_matrix

# Assuming similarity and utility matrix functions from Exercises 3 and 4 are available
# Define the rating estimation function
def estimate_rating(user_id, movie_id, rated_by, user_col, neighborhood_size=10):
    """Estimate rating of a user on an item based on collaborative filtering."""
    # Check if the movie has been rated by other users
    if movie_id not in rated_by:
        return np.nan  # No ratings for this item

    # Get users who rated the movie
    users_who_rated = rated_by[movie_id]

    # Calculate similarities between the target user and users who rated the movie
    target_user_ratings = user_col[user_id].toarray().flatten()
    similarities = np.array([
        fast_centered_cosine_sim(target_user_ratings, user_col[other_user].toarray().flatten())
        for other_user in users_who_rated
    ])

    # Select the top `neighborhood_size` most similar users
    top_neighbors = np.argsort(similarities)[-neighborhood_size:]
    best_users = users_who_rated[top_neighbors]
    best_similarities = similarities[top_neighbors]

    # Calculate the weighted average of ratings by similar users
    ratings = np.array([user_col[user, movie_id] for user in best_users])
    weighted_sum = np.dot(best_similarities, ratings)
    sum_of_weights = np.sum(np.abs(best_similarities))

    # If no valid neighbors found
    if sum_of_weights == 0:
        return np.nan
    return weighted_sum / sum_of_weights


Exercise 2

*   testing the test_centered_cosine_sim
*   testing fast_centered_cosine_sim


In [84]:
def test_centered_cosine_sim():
    # Test with linear sequence vectors
    vec1 = csr_matrix([1, 2, 3, 4, 5])
    vec2 = csr_matrix([2, 3, 4, 5, 6])
    print("Linear sequence similarity:", centered_cosine_sim(vec1, vec2))

    # Test with vectors with NaNs
    vec3 = csr_matrix([1, np.nan, 3, np.nan, 5])
    vec4 = csr_matrix([2, np.nan, 4, np.nan, 6])
    print("Sparse vectors with NaNs similarity:", centered_cosine_sim(vec3, vec4))

test_centered_cosine_sim()


Linear sequence similarity: 0.9999999999999998
Sparse vectors with NaNs similarity: 0.9999999999999998


In [117]:
# Test function to verify
def test_fast_centered_cosine_sim():
    utility_matrix = csr_matrix([[1, 0, 3, 4], [0, 2, 1, 5], [2, 1, 0, 0]], dtype=np.float64)
    vector = csr_matrix([5, 3, 1, 4], dtype=np.float64).T  # Column vector
    similarity = fast_centered_cosine_sim(utility_matrix, vector, axis=0)
    print("Similarity with utility matrix:\n", similarity)

# Run the test
test_fast_centered_cosine_sim()

matrix_centered shape: (3, 4)
vector_centered shape after adjustment: (4,)
Similarity with utility matrix:
 [-0.21380899  0.09035079  0.66254135]


Exercise 3

In [132]:
# Load the MovieLens and Lecture datasets
um_movielens = cfd.get_um_by_name(config, "movielens")
um_lecture = cfd.get_um_by_name(config, "lecture_1")

# Rate all items for the lecture toy dataset
all_ratings = rate_all_items(um_lecture, 4, 2)
print ("all_ratings lecture toy dataset:", all_ratings)

# Rate all items the MovieLens data
all_ratings_movielens = rate_all_items(um_movielens, 0, 2)
print("all_ratings_movielens:", all_ratings_movielens)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
item_idx: 4787, neighbors: [186 284], rating: 1.611591100692749
item_idx: 4788, neighbors: [547 186], rating: -2.3287205696105957
item_idx: 4789, neighbors: [468 476], rating: 3.002462863922119
item_idx: 4790, neighbors: [312 457], rating: 0.8593679070472717
item_idx: 4791, neighbors: [547 186], rating: -3.164360523223877
item_idx: 4792, neighbors: [227 476], rating: 2.0
item_idx: 4793, neighbors: [186 344], rating: -0.050077054649591446
item_idx: 4794, neighbors: [331 186], rating: -2.2455532550811768
item_idx: 4795, neighbors: [547 186], rating: -2.7465405464172363
item_idx: 4796, neighbors: [186], rating: -4.0
item_idx: 4797, neighbors: [646 550], rating: 3.1146881580352783
item_idx: 4798, neighbors: [186], rating: -4.0
item_idx: 4799, neighbors: [227 356], rating: 3.038051128387451
item_idx: 4800, neighbors: [186 312], rating: -0.11779474467039108
item_idx: 4801, neighbors: [696 186], rating: -0.6919110417366028
ite

 Exercise 4

In [83]:
# Run the function to create the structures
rated_by, user_col = create_sparse_structures_with_tf()

# Sample output to verify
print("Sample rated_by for item 0:", rated_by[0] if len(rated_by) > 0 else [])
print("Sample user_col for user 0:", user_col[0].toarray() if len(user_col) > 0 else "No ratings")


Loaded dataset 'movielens/100k' with 100000 ratings and features: FeaturesDict({
    'bucketized_user_age': float32,
    'movie_genres': Sequence(ClassLabel(shape=(), dtype=int64, num_classes=21)),
    'movie_id': string,
    'movie_title': string,
    'raw_user_age': float32,
    'timestamp': int64,
    'user_gender': bool,
    'user_id': string,
    'user_occupation_label': ClassLabel(shape=(), dtype=int64, num_classes=22),
    'user_occupation_text': string,
    'user_rating': float32,
    'user_zip_code': string,
})
Filtering tf dataset for user_id, movie_id and user_rating
Creating a vocabulary for user_id (str -> int)
Vocabulary of user_id's has size: 944
Creating a vocabulary for movie_id (str -> int)
Vocabulary of movie_id's has size: 1683
Sample rated_by for item 0: []
Sample user_col for user 0: [[0. 0. 0. ... 0. 0. 0.]]


Exercise 5

In [130]:

# Part (b): User-item pairs to test
test_pairs = [
    (828, 11), (2400, 4725), (3765, 1270), (4299, 4020), (5526, 2432),
    (6063, 4525), (7045, 4100), (8160, 6300), (9682, 1212), (10277, 7355)
]

# Run the estimation for each pair and print results
for idx, (user_id, movie_id) in enumerate(test_pairs):
    rating_estimate = cfa.estimate_rating(user_id, movie_id, rated_by, user_col)
    print(f"Estimated rating for user {user_id} on movie {movie_id}: {rating_estimate}")

# Part (c): Memory usage monitoring for the first six pairs
memory_usages = []
for user_id, movie_id in test_pairs[:6]:
    process = psutil.Process()
    process.memory_info()  # Reset memory info

    rating_estimate = cfa.estimate_rating(user_id, movie_id, rated_by, user_col)

    max_memory = process.memory_info().rss / (1024 ** 2)  # Convert to MB
    memory_usages.append(max_memory)
    print(f"Max memory usage for user {user_id} on movie {movie_id}: {max_memory:.2f} MB")

# Report max memory usage
print("Memory usage for first 6 user-item pairs:", memory_usages)



Estimated rating for user 828 on movie 11: nan
Estimated rating for user 2400 on movie 4725: nan
Estimated rating for user 3765 on movie 1270: nan
Estimated rating for user 4299 on movie 4020: nan
Estimated rating for user 5526 on movie 2432: nan
Estimated rating for user 6063 on movie 4525: nan
Estimated rating for user 7045 on movie 4100: nan
Estimated rating for user 8160 on movie 6300: nan
Estimated rating for user 9682 on movie 1212: nan
Estimated rating for user 10277 on movie 7355: nan
Max memory usage for user 828 on movie 11: 1817.01 MB
Max memory usage for user 2400 on movie 4725: 1817.01 MB
Max memory usage for user 3765 on movie 1270: 1817.01 MB
Max memory usage for user 4299 on movie 4020: 1817.01 MB
Max memory usage for user 5526 on movie 2432: 1817.01 MB
Max memory usage for user 6063 on movie 4525: 1817.01 MB
Memory usage for first 6 user-item pairs: [1817.01171875, 1817.01171875, 1817.01171875, 1817.01171875, 1817.01171875, 1817.01171875]
